# PPO 教学（手写公式 + 最小实现，无 LoRA）

本 Notebook 目标：用尽量少的现成“PPO/RLHF Trainer”库（不使用 TRL），用 PyTorch 手写 PPO 的关键计算，并让代码变量名能对应公式。

注意：

- 这是教学最小实现：只做单卡、超小 batch、toy reward（规则奖励），便于看懂每一步。
- “无 LoRA”表示会更新基座模型全部参数，显存要求最高；如果显存不够，请用后面的 LoRA 版。


## 1. PPO（token 级）核心公式

把语言模型生成看成一个序列决策过程：

- prompt 为 $x$，模型生成 response $y=(a_1,\dots,a_T)$
- 每个 token $a_t$ 是一步 action；状态 $s_t=(x,a_{<t})$
- 训练策略（actor）为 $\pi_\theta$；rollout 时的旧策略为 $\pi_{\theta_{old}}$
- 参考策略（冻结，用于 KL 约束）为 $\pi_{ref}$
- 价值函数（critic）为 $V_\theta(s_t)$（这里用一个 value head 预测）

### 1.1 token 对数概率（实现里会显式算 log_softmax + gather）

$$
\log \pi_\theta(y\mid x)=\sum_{t=1}^{T} \log \pi_\theta(a_t\mid s_t)
$$

### 1.2 KL 约束（用采样动作的无偏估计）

$$
\widehat{KL}_t = \log\pi_\theta(a_t\mid s_t)-\log\pi_{ref}(a_t\mid s_t)
$$

常见做法是把 KL 作为 shaping reward（每个 token 一个惩罚）：

$$
r_t^{KL}=-\lambda_{KL}\,\widehat{KL}_t
$$

并在最后一步加上奖励模型/规则奖励 $r_{rm}(x,y)$：

$$
r_T \leftarrow r_T + r_{rm}(x,y)
$$

### 1.3 GAE 优势函数（Generalized Advantage Estimation）

$$
\delta_t = r_t + \gamma V(s_{t+1}) - V(s_t)
$$

$$
A_t = \delta_t + \gamma\lambda\,A_{t+1}
$$

$$
R_t = A_t + V(s_t)
$$

### 1.4 PPO clipped objective（策略更新的核心）

$$
\rho_t(\theta)=\frac{\pi_\theta(a_t\mid s_t)}{\pi_{\theta_{old}}(a_t\mid s_t)}=\exp(\log\pi_\theta-\log\pi_{\theta_{old}})
$$

$$
L^{clip}(\theta)=\mathbb{E}_t\left[\min\left(\rho_t A_t,\; \mathrm{clip}(\rho_t,1-\epsilon,1+\epsilon)A_t\right)\right]
$$

### 1.5 Value loss（常见也会做 value clipping）

$$
L^V=\frac12\,\mathbb{E}_t\left[\max\Big((V_\theta-R_t)^2,\; (\mathrm{clip}(V_\theta, V_{old}\pm\epsilon_V)-R_t)^2\Big)\right]
$$

### 1.6 熵奖励（可选，鼓励探索）

$$
H(\pi(\cdot\mid s_t))=-\sum_a \pi(a\mid s_t)\log\pi(a\mid s_t)
$$

### 1.7 总 loss（最小化）

$$
\mathcal{L}= -L^{clip} + c_V L^V - c_{ent}\,\mathbb{E}_t[H]
$$

下面代码会用同名变量（`logpi_old/logpi_ref/kl/rewards/advantages/ratio/...`）逐项实现。


## 2. 环境与模型加载（建议离线）

- 建议 `conda activate llm`
- 默认优先用 `MODELSCOPE_CACHE` 下的本地模型目录（避免联网）
- 参考策略 $\pi_{ref}$ 会放到 CPU（更省显存，速度会慢一些，但教学足够）


In [ ]:
import os
import random
import sys
from dataclasses import dataclass
from pathlib import Path
from typing import Any, Dict, List, Tuple

import torch
import torch.nn.functional as F
from torch.nn.utils import clip_grad_norm_
from transformers import AutoModelForCausalLM, AutoTokenizer

os.environ.setdefault("MODELSCOPE_CACHE", r"D:/myProject/modelscope_hub")
print("python:", sys.executable)
print("torch:", torch.__version__, "cuda:", torch.cuda.is_available())
print("MODELSCOPE_CACHE:", os.environ["MODELSCOPE_CACHE"])

seed = 42
random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

device = "cuda" if torch.cuda.is_available() else "cpu"
device


In [ ]:
# 选择模型（优先本地缓存目录）
local_dir = Path(os.environ["MODELSCOPE_CACHE"]) / "models" / "qwen" / "Qwen2-0___5B-Instruct"
model_name_or_path = str(local_dir) if local_dir.exists() else "qwen/Qwen2-0.5B-Instruct"
print("model_name_or_path:", model_name_or_path)

# dtype
if device == "cuda":
    dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
else:
    dtype = torch.float32
print("dtype:", dtype)

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = "<|endoftext|>"
tokenizer.padding_side = "left"  # 生成时更方便

SYSTEM_PROMPT = "You are a helpful assistant."

class ActorCritic(torch.nn.Module):
    def __init__(self, base: torch.nn.Module):
        super().__init__()
        self.base = base
        self.value_head = torch.nn.Linear(base.config.hidden_size, 1)

    def forward(self, input_ids: torch.Tensor, attention_mask: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        out = self.base(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True,
            use_cache=False,
            return_dict=True,
        )
        hidden = out.hidden_states[-1]  # (B, L, H)
        values = self.value_head(hidden).squeeze(-1)  # (B, L)
        return out.logits, values

# 策略模型（可训练）
actor_base = AutoModelForCausalLM.from_pretrained(model_name_or_path, torch_dtype=dtype).to(device)
actor_base.config.use_cache = False
actor_base.config.pad_token_id = tokenizer.pad_token_id

if hasattr(actor_base, "gradient_checkpointing_enable") and device == "cuda":
    actor_base.gradient_checkpointing_enable()  # 省显存

actor_critic = ActorCritic(actor_base).to(device)
actor_critic.value_head.to(device=device, dtype=dtype)

# 参考策略 π_ref（冻结，放 CPU 省显存）
ref_device = "cpu"
ref_model = AutoModelForCausalLM.from_pretrained(model_name_or_path, torch_dtype=torch.float32).to(ref_device)
ref_model.eval()
for p in ref_model.parameters():
    p.requires_grad_(False)

print("ready")


## 3. Toy 任务与规则奖励 $r_{rm}(x,y)$

真实 RLHF 用 reward model（RM）。教学版这里用规则奖励：检查输出是否包含期望字符串。

奖励设计（可自行改）：

- 命中期望：+1
- 不命中：-1
- 额外长度：轻微惩罚（鼓励更短）


In [ ]:
train_tasks = [
    {"prompt": "请只回答一个词：小鱼儿", "expected": "小鱼儿"},
    {"prompt": "请只输出数字 4，不要额外文字。2+2等于几？", "expected": "4"},
    {"prompt": "把“我喜欢机器学习”翻译成英文，只输出翻译。", "expected": "I like machine learning"},
    {"prompt": "请只回答：通义千问", "expected": "通义千问"},
]

def normalize_text(s: str) -> str:
    s = s.strip().lower()
    for ch in [" ", "\n", "\t", "。", "，", ",", ".", "!", "?", "：", ":", "\"", "'"]:
        s = s.replace(ch, "")
    return s

def rule_reward(prompt: str, response: str, expected: str) -> float:
    resp = normalize_text(response)
    exp = normalize_text(expected)
    hit = exp in resp
    base = 1.0 if hit else -1.0
    length_penalty = 0.002 * len(resp)
    return base - length_penalty

train_tasks[0]


## 4. 关键实现：logprob / KL / GAE / PPO loss

下面函数会尽量贴近公式：

- `action_logp = log π(a_t|s_t)`：用 `log_softmax(logits)` + `gather` 取出采样 action 的对数概率
- `kl = logpi - logref`
- `rewards = -kl_coef * kl`，最后一个 token 加上 `rm_reward`
- GAE：按 $\delta_t$ 和 $A_t$ 递推
- PPO：按 ratio + clip objective


In [ ]:
@torch.inference_mode()
def build_prompt_input_ids(tok: Any, user_prompt: str) -> torch.Tensor:
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_prompt},
    ]
    text = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    enc = tok(text, return_tensors="pt")
    return enc["input_ids"][0]

@torch.inference_mode()
def sample_response(
    m: torch.nn.Module,
    tok: Any,
    prompt_input_ids: torch.Tensor,
    max_new_tokens: int = 48,
    temperature: float = 1.0,
    top_p: float = 0.9,
) -> Tuple[torch.Tensor, torch.Tensor]:
    m.eval()
    input_ids = prompt_input_ids.unsqueeze(0).to(device)

    old_use_cache = getattr(m.config, "use_cache", True)
    m.config.use_cache = True
    out = m.generate(
        input_ids=input_ids,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
        eos_token_id=tok.eos_token_id,
        pad_token_id=tok.pad_token_id,
    )
    m.config.use_cache = old_use_cache

    full_ids = out[0].detach().cpu()
    response_ids = full_ids[prompt_input_ids.numel() :]
    return full_ids, response_ids

def action_logprobs_from_logits(
    logits: torch.Tensor,
    input_ids: torch.Tensor,
    prompt_len: int,
    response_len: int,
) -> Tuple[torch.Tensor, torch.Tensor]:
    """返回 (action_logp, entropy)。
    
    - logits: (1, L, V)
    - input_ids: (1, L)
    - action token 是 input_ids[prompt_len : prompt_len+response_len]
    - logprob 用 logits 在 action token 前一个位置（shift）
    """
    log_probs = F.log_softmax(logits, dim=-1)
    positions = torch.arange(prompt_len, prompt_len + response_len, device=logits.device)
    logp_positions = positions - 1
    action_ids = input_ids[0, positions]
    action_logp = log_probs[0, logp_positions, :].gather(dim=-1, index=action_ids.unsqueeze(-1)).squeeze(-1)

    # 熵：H = -sum p log p
    step_log_probs = log_probs[0, logp_positions, :]
    entropy = -(step_log_probs.exp() * step_log_probs).sum(dim=-1)
    return action_logp, entropy

def get_policy_logp_value_entropy(
    ac: ActorCritic,
    full_ids: torch.Tensor,
    prompt_len: int,
    response_len: int,
    use_grad: bool,
) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    ctx = torch.enable_grad() if use_grad else torch.inference_mode()
    with ctx:
        input_ids = full_ids.unsqueeze(0).to(device)
        attention_mask = torch.ones_like(input_ids, dtype=torch.long)
        logits, values_all = ac(input_ids=input_ids, attention_mask=attention_mask)
        action_logp, entropy = action_logprobs_from_logits(logits, input_ids, prompt_len, response_len)
        positions = torch.arange(prompt_len, prompt_len + response_len, device=device)
        value_positions = positions - 1
        values = values_all[0, value_positions]
        return action_logp, values, entropy

@torch.inference_mode()
def get_ref_logp(
    ref: torch.nn.Module,
    full_ids: torch.Tensor,
    prompt_len: int,
    response_len: int,
) -> torch.Tensor:
    input_ids = full_ids.unsqueeze(0).to(ref_device)
    attention_mask = torch.ones_like(input_ids, dtype=torch.long)
    logits = ref(input_ids=input_ids, attention_mask=attention_mask, use_cache=False, return_dict=True).logits
    log_probs = F.log_softmax(logits, dim=-1)
    positions = torch.arange(prompt_len, prompt_len + response_len, device=logits.device)
    logp_positions = positions - 1
    action_ids = input_ids[0, positions]
    logp = log_probs[0, logp_positions, :].gather(dim=-1, index=action_ids.unsqueeze(-1)).squeeze(-1)
    return logp.to(device)

def compute_gae(rewards: torch.Tensor, values: torch.Tensor, gamma: float, gae_lambda: float) -> Tuple[torch.Tensor, torch.Tensor]:
    # rewards/values: (T,)
    T = rewards.shape[0]
    advantages = torch.zeros_like(rewards)
    last_gae = torch.zeros((), device=rewards.device, dtype=rewards.dtype)
    for t in reversed(range(T)):
        next_value = values[t + 1] if t < T - 1 else torch.zeros((), device=values.device, dtype=values.dtype)
        delta = rewards[t] + gamma * next_value - values[t]
        last_gae = delta + gamma * gae_lambda * last_gae
        advantages[t] = last_gae
    returns = advantages + values
    return advantages, returns


## 5. PPO 训练循环（单样本/超小 batch 教学版）

流程（对应 PPO 标准做法）：

1) rollout：用当前策略采样 response，得到 `logpi_old`、`values_old`、`logpi_ref`
2) 构造 shaped rewards：`rewards = -kl_coef * (logpi_old - logpi_ref)`，并在最后一个 token 加上规则奖励 `rm_reward`
3) 用 GAE 得到 `advantages/returns`（作为固定的训练目标）
4) PPO update：对同一条轨迹做若干 epoch 的 clipped 更新


In [ ]:
# PPO 超参（教学版默认值）
train_steps = 20
max_new_tokens = 48

kl_coef = 0.05
gamma = 1.0
gae_lambda = 0.95

clip_eps = 0.2
value_clip_eps = 0.2
vf_coef = 0.5
ent_coef = 0.0
ppo_epochs = 2
max_grad_norm = 1.0

lr = 2e-6  # 无 LoRA 全参更新，建议很小
optimizer = torch.optim.AdamW(actor_critic.parameters(), lr=lr)

use_amp = device == "cuda"
use_bf16 = use_amp and torch.cuda.is_bf16_supported()
autocast_dtype = torch.bfloat16 if use_bf16 else torch.float16
autocast_device_type = "cuda" if use_amp else "cpu"
scaler = torch.cuda.amp.GradScaler(enabled=use_amp and not use_bf16)

def ppo_update_one_trajectory(
    full_ids: torch.Tensor,
    prompt_len: int,
    response_len: int,
    logpi_old: torch.Tensor,
    values_old: torch.Tensor,
    advantages: torch.Tensor,
    returns: torch.Tensor,
) -> Dict[str, float]:
    actor_critic.train()
    advantages = (advantages - advantages.mean()) / (advantages.std().clamp_min(1e-8))

    metrics: Dict[str, float] = {}
    for epoch in range(ppo_epochs):
        with torch.autocast(device_type=autocast_device_type, dtype=autocast_dtype, enabled=use_amp):
            logpi, values, entropy = get_policy_logp_value_entropy(
                actor_critic, full_ids, prompt_len, response_len, use_grad=True
            )

            # ratio = exp(logπ_new - logπ_old)
            ratio = torch.exp(logpi - logpi_old)

            # L_clip = mean(min(ratio*A, clip(ratio)*A))
            surr1 = ratio * advantages
            surr2 = torch.clamp(ratio, 1.0 - clip_eps, 1.0 + clip_eps) * advantages
            policy_loss = -torch.min(surr1, surr2).mean()

            # value loss（带 clipping）
            values_clipped = values_old + (values - values_old).clamp(-value_clip_eps, value_clip_eps)
            v_loss1 = (values - returns) ** 2
            v_loss2 = (values_clipped - returns) ** 2
            value_loss = 0.5 * torch.max(v_loss1, v_loss2).mean()

            entropy_loss = -entropy.mean()  # maximize entropy

            loss = policy_loss + vf_coef * value_loss + ent_coef * entropy_loss

        optimizer.zero_grad(set_to_none=True)
        if scaler.is_enabled():
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            clip_grad_norm_(actor_critic.parameters(), max_grad_norm)
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            clip_grad_norm_(actor_critic.parameters(), max_grad_norm)
            optimizer.step()

        approx_kl = (logpi_old - logpi).mean().detach()
        clip_frac = ((ratio - 1.0).abs() > clip_eps).float().mean().detach()

        metrics = {
            "loss": float(loss.detach().cpu().item()),
            "policy_loss": float(policy_loss.detach().cpu().item()),
            "value_loss": float(value_loss.detach().cpu().item()),
            "entropy": float(entropy.mean().detach().cpu().item()),
            "approx_kl": float(approx_kl.cpu().item()),
            "clip_frac": float(clip_frac.cpu().item()),
        }

    return metrics

@torch.inference_mode()
def rollout_one(prompt: str, expected: str) -> Dict[str, Any]:
    prompt_ids = build_prompt_input_ids(tokenizer, prompt)
    full_ids, response_ids = sample_response(actor_critic.base, tokenizer, prompt_ids, max_new_tokens=max_new_tokens)
    response_text = tokenizer.decode(response_ids, skip_special_tokens=True)

    prompt_len = prompt_ids.numel()
    response_len = int(response_ids.numel())
    if response_len == 0:
        return {"skip": True, "prompt": prompt, "response": response_text}

    rm_reward = rule_reward(prompt, response_text, expected)

    logpi_old, values_old, _ = get_policy_logp_value_entropy(
        actor_critic, full_ids, prompt_len, response_len, use_grad=False
    )
    logpi_ref = get_ref_logp(ref_model, full_ids, prompt_len, response_len)

    kl = logpi_old - logpi_ref
    rewards = -kl_coef * kl
    rewards[-1] = rewards[-1] + torch.tensor(rm_reward, device=device, dtype=rewards.dtype)

    advantages, returns = compute_gae(rewards, values_old, gamma=gamma, gae_lambda=gae_lambda)

    return {
        "skip": False,
        "prompt": prompt,
        "expected": expected,
        "response": response_text,
        "full_ids": full_ids,
        "prompt_len": prompt_len,
        "response_len": response_len,
        "rm_reward": float(rm_reward),
        "kl_mean": float(kl.mean().detach().cpu().item()),
        "logpi_old": logpi_old,
        "values_old": values_old,
        "advantages": advantages.detach(),
        "returns": returns.detach(),
    }

print("ready for training")


In [ ]:
for step in range(train_steps):
    task = random.choice(train_tasks)
    rollout = rollout_one(task["prompt"], task["expected"])
    if rollout["skip"]:
        print(f"step={step} skip(empty response)")
        continue

    metrics = ppo_update_one_trajectory(
        full_ids=rollout["full_ids"],
        prompt_len=rollout["prompt_len"],
        response_len=rollout["response_len"],
        logpi_old=rollout["logpi_old"],
        values_old=rollout["values_old"],
        advantages=rollout["advantages"],
        returns=rollout["returns"],
    )

    print(
        f"step={step} rm_reward={rollout['rm_reward']:.3f} kl_mean={rollout['kl_mean']:.3f} "
        f"loss={metrics['loss']:.4f} policy={metrics['policy_loss']:.4f} value={metrics['value_loss']:.4f} "
        f"approx_kl={metrics['approx_kl']:.4f} clip_frac={metrics['clip_frac']:.3f}"
    )
    print("prompt:", rollout["prompt"])
    print("response:", rollout["response"])
    print("-" * 60)


## 6. 简单验证

随机抽一个 prompt，看模型现在的输出是否更贴近规则奖励偏好。


In [ ]:
@torch.inference_mode()
def chat(prompt: str, max_new_tokens: int = 64) -> str:
    prompt_ids = build_prompt_input_ids(tokenizer, prompt)
    full_ids, response_ids = sample_response(actor_critic.base, tokenizer, prompt_ids, max_new_tokens=max_new_tokens, temperature=0.7, top_p=0.9)
    return tokenizer.decode(response_ids, skip_special_tokens=True)

for t in train_tasks:
    print("Q:", t["prompt"])
    print("A:", chat(t["prompt"]))
    print()
